라라랜드 영화 리뷰들을 학습데이터로 이용합니다.

In [1]:
import config
from navermovie_comments import load_movie_comments

_, texts, _ = load_movie_comments(idxs='134963')

Character-level RNN 을 이용합니다. RNN 은 character embedding matrix 를 가지고 있습니다. 이 크기를 지정하기 위해 데이터의 character 종류를 확인합니다. `scan_vocabulary` 함수는 한 문장이 list of str 임을 가정하지만, str 역시 list of char 이기 때문에 scan_vocabulary 함수를 이용할 수 있습니다.

In [2]:
from utils import scan_vocabulary

class TextsDecorator:
    def __init__(self, texts):
        self.texts = texts
    def __iter__(self):
        for text in self.texts:
            yield text.replace(' ','')

decorated_texts = TextsDecorator(texts)
idx_to_char, char_to_idx = scan_vocabulary(decorated_texts, min_count=1)
print('num characters = %d' % len(idx_to_char))

num characters = 1609


to_idx 는 character 를 index 로 변환하는 함수입니다. 모르는 글자는 char_to_idx 길이의 값으로 표현하도록 했습니다. Character size + 1 의 값을 Embedding layer 의 길이로 정의할 겁니다.

In [3]:
from utils import encode_sequence

print(encode_sequence('라', char_to_idx))
print(encode_sequence('퀡', char_to_idx))

tensor([32])
tensor([1609])


예문을 학습용 x, y 로 만듭니다.

In [4]:
idx_to_char[:10]

['이', '다', '영', '는', '화', '고', '지', '람', '관', '객']

In [5]:
texts[0]

'시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다'

sent_to_xy 함수에 str 과 char_to_idx 를 입력하여 torch.LongTensor 의 character sequence 와 tag sequence 를 만듭니다.

In [6]:
from space import sent_to_xy

sent_to_xy(texts[0], char_to_idx)

(tensor([ 57,  40, 290,  12,  25,  18,   5, 294,  67,  26,   1,  95,   4, 112,
          45, 165, 108,   6, 109,  24,  23,  76,   6,  91,  13,   6, 103,   5,
          45, 123,  67,  26,   1]),
 tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 1]))

학습용 데이터 X, Y 를 만듭니다.

In [7]:
X = [] # list of sentence
Y = [] # list of label

for text in texts:
    x, y = sent_to_xy(text, char_to_idx)
    X.append(x)
    Y.append(y)

set_num_threads 를 설정하면 사용 가능한 코어의 최대 개수를 정의할 수 있습니다. 기본값은 모든 코어를 이용하는 것입니다.

그 외의 모델 패러매터들을 설정합니다. GRU 와 LSTM 은 forward 함수의 return 형식이 다릅니다. 이는 rnnspace.model 파일을 확인하세요.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

from space import GRUSpace

# set parameters
embedding_dim = 16
hidden_dim = 64
tagset_size = 2
vocab_size = len(idx_to_char) + 1 # for unknown character
bidirectional = True

# model
model = GRUSpace(embedding_dim, hidden_dim, vocab_size, tagset_size, bidirectional=bidirectional)

만든 함수를 적용하여 띄어쓰기 교정 결과를 확인합니다.

In [9]:
model(X[0])

tensor([[-0.5543, -0.8545],
        [-0.5405, -0.8734],
        [-0.5967, -0.7999],
        [-0.6115, -0.7820],
        [-0.6448, -0.7439],
        [-0.6635, -0.7237],
        [-0.6247, -0.7667],
        [-0.5131, -0.9128],
        [-0.6033, -0.7918],
        [-0.6247, -0.7666],
        [-0.6354, -0.7545],
        [-0.6612, -0.7262],
        [-0.7511, -0.6384],
        [-0.7641, -0.6269],
        [-0.6991, -0.6872],
        [-0.6176, -0.7749],
        [-0.7037, -0.6827],
        [-0.7444, -0.6444],
        [-0.7385, -0.6497],
        [-0.7059, -0.6806],
        [-0.7205, -0.6665],
        [-0.6463, -0.7423],
        [-0.6833, -0.7031],
        [-0.6173, -0.7752],
        [-0.6075, -0.7868],
        [-0.7092, -0.6774],
        [-0.6738, -0.7129],
        [-0.6659, -0.7212],
        [-0.6459, -0.7427],
        [-0.6336, -0.7565],
        [-0.6252, -0.7661],
        [-0.6321, -0.7581],
        [-0.6339, -0.7561]], grad_fn=<LogSoftmaxBackward>)

In [10]:
from space import correct

sent = '이건진짜좋은영화 라라랜드진짜좋은영화'
print(correct(sent, char_to_idx, model))

이건진짜좋은영화 라라랜드진짜좋은영화


In [11]:
def train(model, X, Y, optimizer, loss_func, epoch):
    for i, (x, y) in enumerate(zip(X, Y)):
        model.zero_grad()
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('\repoch = {}, batch = {} / {}'.format(epoch, i, len(X)), end='')
    print('\repoch {} was done{}'.format(epoch, ' '*60))
    return model

In [14]:
import time

max_epochs = 5
num_threads = 3

model = GRUSpace(embedding_dim, hidden_dim, vocab_size, tagset_size, bidirectional=bidirectional)
loss_func = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# set max num of threads
torch.set_num_threads(num_threads)

print('Before training')
print(correct(sent, char_to_idx, model), end='\n\n')

for epoch in range(1, max_epochs + 1):

    t = time.time()
    model = train(model, X, Y, optimizer, loss_func, epoch)
    t = time.time() - t

    print(correct(sent, char_to_idx, model))
    print('training time = {:.2} seconds'.format(t), end='\n\n')

Before training
이 건진짜좋은 영 화 라 라 랜드진짜좋은 영 화

epoch 1 was done                                                            
이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화
training time = 2.5e+02 seconds

epoch = 2, batch = 3100 / 15599

KeyboardInterrupt: 

In [15]:
sent = '스파이더맨같은모르는글자를넣었으면?'
print(correct(sent, char_to_idx, model))

스파이 더맨같은 모르는 글 자를 넣었으면 ?


In [16]:
sent = '라라랜드영화와관련된말을넣는다면?'
print(correct(sent, char_to_idx, model))

라라랜드 영화와 관련된 말을 넣는 다면 ?
